#Primeiro trabalho prático
##Nome: Diego Fleury Corrêa de Moraes
##nusp: 11800584
###data: 20/10/2021

In [1]:
# Setup inicial (mais geral)

#imports
import numpy as np
import pandas as pd

#Configuração para usar a base de dados de modo
#eficiente no google colab  --> comentar quando for rodar

from google.colab import drive

drive.mount('/content/drive')

PROJECT_PATH = '/content/drive/MyDrive/Competicao_DS_Arquivos'

!unzip $PROJECT_PATH/house-prices-advanced-regression-techniques.zip


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/drive/MyDrive/Competicao_DS_Arquivos/house-prices-advanced-regression-techniques.zip
replace data_description.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [2]:
 !ls 

data_description.txt  sample_data	     sub.csv   train.csv
drive		      sample_submission.csv  test.csv


In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

target_col = 'SalePrice'

y = pd.DataFrame(train[target_col])
X = train.drop(target_col,axis=1)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    train_size=0.65,
    random_state=42
)

Primeiramente, analiza-se a descrição de todas as features disponíveis. A partir disso será possível notarmos alguns padrões.

In [5]:
!cat data_description.txt

MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

MSZoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM

#Separação de campos de features
À partir de uma análise de todas as features são feitas as seguintes partições dos tipos de grupos em que serão separadas. A escolha de separação leva em conta a noção de que, dadas variáveis próximas em significado (referem-se a um mesmo objeto, ideia ou noção) tem-se alta probabilidade que estão, não somente altamente correlacionadas como também a união destas informações provavelmente é mais útil ao modelo do que cada uma em si.

São adicionadas 3 features mais descritivas novas que, embora possam ser colocadas em outros grupos terão um grupo separado para sua análise/agrupamento. Esta decisão é justificada na medida em que possuem características em comum (serem um tamanho médio de alguma grandeza que geralmente resulta em uma maior valorização do imóvel) e para fins de teste de sua qualidade como noções úteis ou não ao modelo.

Além disso são descartadas algumas features por não serem úteis do ponto de vista prático, como informações da venda (que não estariam disponíveis em uma aplicação real do modelo) e o id (apenas um metadado).

In [6]:
#como, para este dataset, não temos valores nulos de área, e é uma feature 
#que dificilmente do ponto de vista prático estaria em falta, 
#descarta-se a necessidade de correção de valores nulos

X_train['LotArea'].isna().sum(), X_test['LotArea'].isna().sum()

(0, 0)

##Questão 1

###a) Proponha 3 novas variáveis explicativas que façam sentido do ponto de vista prático do problema;

In [ ]:
#Criacao de features novas
#train
total_area_train = X_train['LotArea'] + X_train['1stFlrSF'] + X_train['2ndFlrSF']
X_train['AvgAbvGrLivArea'] = X_train['GrLivArea']/total_area_train
X_train['AvgRmsAbvGrdSize'] = X_train['TotRmsAbvGrd']/total_area_train
X_train['AvgBedroomAbvGrSize'] = X_train['BedroomAbvGr']/total_area_train

#test
total_area_test = X_test['LotArea'] + X_test['1stFlrSF'] + X_test['2ndFlrSF']
X_test['AvgAbvGrLivArea'] = X_test['GrLivArea']/total_area_test
X_test['AvgRmsAbvGrdSize'] = X_test['TotRmsAbvGrd']/total_area_test
X_test['AvgBedroomAbvGrSize'] = X_test['BedroomAbvGr']/total_area_test

#Questão 2
###b) Faça uso de uma técnica de modelagem ainda não falada na disciplina. 
Essa técnica não precisa ser uma modelo de regressão, ela pode ser uma técnica
de agrupamento para ajudar na criação de novas variáveis. Considere como
técnicas já vistas: **KNN, árvores e ensembles**.

* (Usada KMeans para agrupamento de noções similares, sob a hipótese de alta correlação entre elas)

In [8]:
#features novas agrupadas: 3
avgs = [
        'AvgAbvGrLivArea', 'AvgRmsAbvGrdSize', 'AvgBedroomAbvGrSize'
]

In [9]:
#features pouco (ou não) relevantes ao problema: -5
drop = [
'Id', 'MoSold', 'YrSold','SaleType',
'SaleCondition'    
]

#features relacionadas à garagem: 8
garage = ["GarageType",'GarageYrBlt', 'GarageFinish',
        'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond','PavedDrive'
]

#features relacionadas ao porão: 11
bsmt = ['BsmtQual', 'BsmtCond',
        'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
        'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
            'BsmtFullBath', 'BsmtHalfBath'
]

#features relacinadas a valores considerados (de certa forma) "de luxo": 12
extras = ['WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch',
          'PoolArea', 'PoolQC', 'Fence',
        'MiscFeature', 'MiscVal','Fireplaces','FireplaceQu'
]

#features relacionadas a localizacao: 6
local = ['MSZoning','Alley','Street',
         'Neighborhood', 'Condition1','Condition2'
]

#features relacionadas a infraestrutura da casa em geral: 14
infra = ['Utilities','OverallQual', 'OverallCond','YearBuilt','YearRemodAdd',
            'ExterQual', 'ExterCond', 'Foundation',
                  'Heating', 'HeatingQC','CentralAir', 'Electrical',
         'LowQualFinSF','Functional'
]

#features relacionadas ao terreno e estilo da casa em si : 18
terreno_estilo= ['MSSubClass', 'LotFrontage', 'LotArea',
                    'LotShape', 'LandContour', 'LotConfig','LandSlope',
                    'BldgType', 'HouseStyle','YearBuilt', 'YearRemodAdd', 
                     'Exterior1st', 'Exterior2nd',
                     'MasVnrType', 'MasVnrArea','1stFlrSF', '2ndFlrSF',
                    'GrLivArea'
]

#features relacionadas a banheiros: 2
bathroom = [
      'FullBath','HalfBath'
]

#features relacionadas a quartos: 2
bedroom = [
   'TotRmsAbvGrd', 'BedroomAbvGr'
]

#features relacionadas a cozinha: 2
kitchen = [
    'KitchenQual','KitchenAbvGr'
]

#features relacionadas ao telhado: 2
roof = [
    'RoofMatl', 'RoofStyle'
]

grupos = [garage,bsmt,extras,local,
          infra,terreno_estilo,bathroom,
          bedroom,kitchen,roof,avgs]

grupos_nomes = ['garage','bsmt','extras','local',
          'infra','terreno_estilo','bathroom',
          'bedroom','kitchen','roof','avgs']

#Criação de features novas
Inicialmente, temos de fazer a separação de variáveis numéricas e categóricas. 
Embora isto possa ser feito de outra maneira, após uma investigação mais a fundo, o procedimento não é tão simples quanto apenas separar em dtypes numéricos e categóricos, pois temos situações em que valores numéricos na verdade, quando vistos pela descrição, são categóricos, enquanto alguns valores categóricos são na verdade ordinais, os quais correspondem a uma noção numérica. 
Exemplos:

In [10]:
#esta feature apenas identifica o "tipo de habitação envolvida na venda", porém possui uma codificação por números inteiros
print(X_train['MSSubClass'].dtypes)

#esta feature é categórica porém indica em sua descrição um nível de regularidade do terreno, sendo no fundo uma noção ordinal, que é inerentemente numérica
print(X_train['LotShape'].dtypes)

int64
object


Para isto, são criadas algumas funções para este processamento. O tratamento matemático escolhido é o agrupamento das features numéricas de modo que estejam **normais** (StandardScaler), um simples tratamento **esparso** na forma de OneHotEncoding para as features puramente categóricas e, enfim, a tentativa de captura uma noção de **ordem relativa** à máxima (100% = 1) nas features ordinais, sendo inicialmente feita a codificação da ordem encontrada (OrdinalEncoder) e, enfim, a mudança de escala para unitária.

As funções implementadas para este fim estão mostradas logo abaixo:

#Importação de bibliotecas úteis ao problema

In [11]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.impute import SimpleImputer

Imputamento de valores é feito através de chamada a uma função separada

In [12]:
def imputa_valores(df_grupo,feat_num,feat_cat,feat_ord):

    if(len(feat_num)!=0):
      #tratamento de valores numericos
      imp_mean = SimpleImputer(missing_values=np.NaN, strategy='mean')
      df_grupo[feat_num] = imp_mean.fit_transform(df_grupo[feat_num])
    
    if(len(feat_cat)!=0):
      #tratamento de valores categoricos
      df_grupo[feat_cat] = df_grupo[feat_cat].fillna('Unk')
    
    if(len(feat_ord)!=0):
      #tratamento de valores ordinais
      imp_mode = SimpleImputer(missing_values=np.NaN,strategy='most_frequent')
      df_grupo[feat_ord] =  imp_mode.fit_transform(df_grupo[feat_ord])

#Função responsável pelo 'fit' do agrupamento

In [13]:
def agrupamento_fit(df_grupo,feat_num,feat_cat,feat_ord,ordem):
  '''
  Esta funcao realiza o agrupamento proposto para o preprocessamento
  dos dados, fazendo a distincao entre variaveis numericas, categoricas
  e ordinais, de forma que dados:

  *Numericos sao normalizados

  *Categoricos sao codificados em OneHotEncoder

  *Ordinais sao codificados em OrdinalEncoder

  '''  

  #Imputamento
  imputa_valores(df_grupo,feat_num,feat_cat,feat_ord)

  if(len(feat_num)!=0):

    #tratamento numerico dado
    ss_grupo = StandardScaler()
    ss_grupo.fit(df_grupo[feat_num])

  else:
    ss_grupo = False

  if(len(feat_cat)!=0):

    #tratamento numerico dado
    ohe_grupo = OneHotEncoder(sparse=False,handle_unknown='ignore')
    ohe_grupo.fit(df_grupo[feat_cat])

  else:
    ohe_grupo = False

  if(len(feat_ord)!=0):

    #tratamento numerico dado
    oe_grupo = OrdinalEncoder(categories=ordem)
    oe_grupo.fit(df_grupo[feat_ord])

  else:
    oe_grupo = False

  return (ss_grupo,ohe_grupo,oe_grupo)

#Função responsável pelo 'transform' do agrupamento

In [14]:
def agrupamento_transform(df_grupo, feat_num, feat_cat, feat_ord, max_ord,
                          ss_grupo, ohe_grupo, oe_grupo):
  '''
  Esta funcao recebe os estimators obtidos em agrupamento fit e transforma os valores baseados
  na operação desejada, ao final retornando os tres tipos concatenados
  '''

  #criar variavel de saida boba, como uma coluna de zeros que sera
  #dropada ao final

  imputa_valores(df_grupo,feat_num,feat_cat,feat_ord)

  #pequeno truque introduzido para condicionalmente poder agregar valores
  feat_grupo = np.zeros((len(df_grupo),1))

  if(ss_grupo != False):
    num_grupo = ss_grupo.transform(df_grupo[feat_num])
    feat_grupo = np.append(feat_grupo,num_grupo, axis=1)

  if(ohe_grupo != False):
    cat_grupo = ohe_grupo.transform(df_grupo[feat_cat])
    feat_grupo = np.append(feat_grupo,cat_grupo, axis=1)

  #max_ord é um vetor dos maiores valores por coluna (ou seja, por feature
  #ordinal). Efetuada a divisão desta forma, um vetor do tipo [[1,1],[2,2]]
  #"dividido" por um vetor do tipo [1,2] resulta em [[1,1],[1,1]]
  if(oe_grupo != False):
    ord_grupo = oe_grupo.transform(df_grupo[feat_ord]) / max_ord
    feat_grupo = np.append(feat_grupo,ord_grupo, axis=1)

  #ao final dar drop na 1a coluna do numpy array (boba)
  feat_grupo = np.delete(feat_grupo, 0, 1)

  return feat_grupo

Nota: a escolha de alcance entre 0 e 1 para categóricas (puras ou ordinais) e com *possibilidade* de extrapolação de escala para numéricas fora feita de forma para que, em média, as features estejam entre 0 e 1 (pois isso é útil para KMeans) porém em casos que possuem outliers numéricos essas relações sejam mais realçadas.

#Função responsável pela criação 'fit' das novas features agrupadas

* Nota: para fins de futura análise, o valor de k também é retornado

In [15]:
def kmeans_features_fit(feat_grupo,k_max=10):
  '''
  Esta funcao recebe os valores retornados por agrupamento_transform e
  realiza a transformacao, retornando o melhor "agrupador" KMean de features
  '''

  silhoutte_list = []
  for k in range(2,k_max+1):
    kmean = KMeans(n_clusters=k)
    kmean.fit(feat_grupo)
    silhoutte_list.append(silhouette_score(feat_grupo, kmean.labels_))

  silhoutte_list = np.array(silhoutte_list)
  k_melhor = 2 + np.argmax(silhoutte_list)
  print("=======================================")
  print("======= Melhor k = ",k_melhor," =======")
  print("=======================================")

  Kmean_Final = KMeans(n_clusters=k_melhor)
  Kmean_Final.fit(feat_grupo)
  return (Kmean_Final,k_melhor)

#Função responsável pelo 'transform' das novas features agrupadas

In [16]:
def k_means_features_transform(KMean_Final,feat_grupo):
  '''
  Esta funcao retorna, finalmente, os valores finais das
  features do grupo
  '''
  final_group_feat = KMean_Final.transform(feat_grupo)
  return final_group_feat

##Esta função tem como objetivo retornar os valores máximos dentro de uma **ORDEM** estabelecida. Isso ocorre pois estamos tentando normalizar a escala das features ordinais entre 0 e 1 com relação à **todos** os valores possíveis, descritos em data_description.txt, não apenas os mínimos e máximos presentes em tempo de treino (por isso não se utiliza MinMaxScaler).

In [17]:
def array_de_normalizacao_das_ordinais(ordem):
  lenghts = []
  for ordenacao in ordem:
    lenghts.append(len(ordenacao))
  return (np.array(lenghts))

Para facilitar a aplicação das funções anteriores, foram criadas as seguintes:

In [18]:
def features_abstraidas_por_grupo_fit_transform(df_grupo,feat_num,feat_cat,feat_ord,ordem,k_max=10):

  ss_grupo, ohe_grupo, oe_grupo = agrupamento_fit(df_grupo,feat_num,feat_cat,feat_ord,ordem)

  feat_grupo= agrupamento_transform(df_grupo,feat_num,feat_cat,feat_ord,array_de_normalizacao_das_ordinais(ordem), ss_grupo,ohe_grupo,oe_grupo)

  KMeanModeloGrupo , k_melhor_grupo = kmeans_features_fit(feat_grupo,k_max)

  final_grupo_feat = k_means_features_transform(KMeanModeloGrupo,feat_grupo)

  return (final_grupo_feat, (ss_grupo, ohe_grupo, oe_grupo, KMeanModeloGrupo), k_melhor_grupo )

def features_abstraidas_por_grupo_transform(df_grupo,feat_num,feat_cat,feat_ord,ordem,ss_grupo, ohe_grupo, oe_grupo, KMeanModeloGrupo):

  feat_grupo = agrupamento_transform(df_grupo,feat_num,feat_cat,feat_ord,array_de_normalizacao_das_ordinais(ordem), ss_grupo,ohe_grupo,oe_grupo)

  final_grupo_feat = k_means_features_transform(KMeanModeloGrupo,feat_grupo)

  return final_grupo_feat

###Para a realização, de forma mais modular, das transformações necessárias, foram utilizados os seguintes dicionários/mapeamentos

In [19]:
#Descrição de todas as features numéricas em cada grupo
dict_num = {
    'garage':['GarageYrBlt','GarageCars','GarageArea'],
    'bsmt':['BsmtFullBath','BsmtHalfBath','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF'],
    'extras':['WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','Fireplaces'],
    'local':[],
    'infra':['YearBuilt', 'YearRemodAdd', 'LowQualFinSF'],
    'terreno_estilo':['MasVnrArea', '1stFlrSF','2ndFlrSF','GrLivArea','LotFrontage','LotArea'],
    'bathroom':['FullBath','HalfBath'],
    'bedroom':['TotRmsAbvGrd', 'BedroomAbvGr'],
    'kitchen':['KitchenAbvGr'],
    'roof':[],
    'avgs': ['AvgAbvGrLivArea', 'AvgRmsAbvGrdSize', 'AvgBedroomAbvGrSize']
}

#Descrição de todas as features categóricas em cada grupo
dict_cat = {
    'garage':['GarageType'],
    'bsmt':[],
    'extras':['MiscFeature'],
    'local':['MSZoning','Neighborhood','Condition1','Condition2'],
    'infra':['Foundation','Heating','Electrical'],
    'terreno_estilo':['MSSubClass','BldgType','HouseStyle','Exterior1st','Exterior2nd','MasVnrType','LotConfig'],
    'bathroom':[],
    'bedroom':[],
    'kitchen':[],
    'roof':['RoofMatl', 'RoofStyle'],
    'avgs':[]
}

#Descrição de todas as features ordinais em cada grupo
dict_ord = {
    'garage':['GarageFinish','GarageQual','GarageCond','PavedDrive'],
    'bsmt':['BsmtExposure','BsmtFinType1','BsmtFinType2','BsmtQual','BsmtCond'],
    'extras':['PoolQC','Fence','FireplaceQu'],
    'local':['Street','Alley'],
    'infra':['Functional','Utilities' ,'OverallQual','OverallCond','ExterQual' ,'ExterCond','HeatingQC','CentralAir'],
    'terreno_estilo':['LotShape','LandContour','LandSlope'],
    'bathroom':[],
    'bedroom':[],
    'kitchen':['KitchenQual'],
    'roof':[],
    'avgs':[]
}

#Descrição da ordem que deve ser seguida nas features ordinais de cada grupo
dict_ordem = {
    'garage':[['NA','Unf','RFn','Fin'],['NA','Po','Fa','TA','Gd','Ex'],['NA','Po','Fa','TA','Gd','Ex'],['N','P','Y']],
    'bsmt':[
              ['NA','No','Mn','Av','Gd'],['NA','Unf','LwQ','Rec','BLQ','ALQ','GLQ'],
              ['NA','Unf','LwQ','Rec','BLQ','ALQ','GLQ'],
              ['NA','Po','Fa','TA','Gd','Ex'],['NA','Po','Fa','TA','Gd','Ex']
           ],
    'extras':[['NA','Fa','TA','Gd','Ex'],['NA','MnWw','GdWo','MnPrv','GdPrv'],['NA','Po','Fa','TA','Gd','Ex']],
    'local':[['Grvl','Pave'],['NA','Grvl','Pave']],
    'infra':[     
              ['Typ','Min1','Min2','Mod','Maj1','Maj2','Sev','Sal'],['ELO','NoSeWa','NoSewr','AllPub'],
              ['1','2','3','4','5','6','7','8','9','10'],['1','2','3','4','5','6','7','8','9','10'],
              ['Po','Fa','TA','Gd','Ex'],['Po','Fa','TA','Gd','Ex'],['Po','Fa','TA','Gd','Ex'],['N','Y']
            ],
    'terreno_estilo':[['Reg','IR1','IR2','IR3'],['Low','HLS','Bnk','Lvl'],['Gtl','Mod','Sev']],
    'bathroom':[],
    'bedroom':[],
    'kitchen':[['Po','Fa','TA','Gd','Ex']],
    'roof': [],
    'avgs':[]    
}

###Como forma escolhida de escolha do número de clusters por agrupamento, nota-se que a entropia é mantida constante pelas transformações, assim, de modo a capturar uma noção simples de conservação da informação do dataset original, limita-se o número total de features (k_total = 80) e escolhem-se a limitação (pois o processo escolhe o melhor baseado na métrica de Silhouette Score) na quantidade de clusters KMeans (k_grupo) baseando-se na proporção da entropia total (H_total) mantida pelos agrupamentos individuais (H_grupo).


In [ ]:
#Drop das variáveis não utilizadas
X_train.drop(drop,axis=1,inplace=True)
X_test.drop(drop,axis=1,inplace=True)

In [21]:
import scipy.stats

def entropia(data):
    p_data = data.value_counts()           
    entropy = scipy.stats.entropy(p_data) 
    return entropy

In [22]:
entropias = []
entropia_total = 0
for i in range(len(grupos)):
  ent_parcial = entropia(X_train[grupos[i]])
  entropia_total+=ent_parcial
  entropias.append(ent_parcial)

proporcoes_por_grupo = np.array(entropias/entropia_total)
print(proporcoes_por_grupo)

[0.16225749 0.16368793 0.         0.04980689 0.15461735 0.16016279
 0.03555755 0.06410313 0.02825312 0.01665573 0.16489802]


In [23]:
def cluster_por_entropia(proporcoes_por_grupo,index,k_original):
  return np.maximum(np.ceil(proporcoes_por_grupo[index]*k_original).astype('int'),2) #maior entre o numero e 2, pois devemos ter ao menos 2 clusters

In [24]:
k_original = 80 #originalmente temos 80 features

k_maximo_por_grupo = {
    'garage':cluster_por_entropia(proporcoes_por_grupo,0,k_original),
    'bsmt':cluster_por_entropia(proporcoes_por_grupo,1,k_original),
    'extras':cluster_por_entropia(proporcoes_por_grupo,2,k_original),
    'local':cluster_por_entropia(proporcoes_por_grupo,3,k_original),
    'infra':cluster_por_entropia(proporcoes_por_grupo,4,k_original),
    'terreno_estilo':cluster_por_entropia(proporcoes_por_grupo,5,k_original),
    'bathroom':cluster_por_entropia(proporcoes_por_grupo,6,k_original),
    'bedroom':cluster_por_entropia(proporcoes_por_grupo,7,k_original),
    'kitchen':cluster_por_entropia(proporcoes_por_grupo,8,k_original),
    'roof':cluster_por_entropia(proporcoes_por_grupo,9,k_original),
    'avgs':cluster_por_entropia(proporcoes_por_grupo,10,k_original)
    
}

In [25]:
k_maximo_por_grupo

{'avgs': 14,
 'bathroom': 3,
 'bedroom': 6,
 'bsmt': 14,
 'extras': 2,
 'garage': 13,
 'infra': 13,
 'kitchen': 3,
 'local': 4,
 'roof': 2,
 'terreno_estilo': 13}

In [ ]:
#Utilização novamente do 'truque' do array 
features_novas_train = np.zeros((len(X_train),1))

modelos = []
melhores_ks = []

for i in range(len(grupos)):
  features_novas_do_grupo, Modelo , k_melhor_grupo = features_abstraidas_por_grupo_fit_transform(
                                                                                  X_train[grupos[i]],
                                                                                  dict_num[grupos_nomes[i]],
                                                                                  dict_cat[grupos_nomes[i]],
                                                                                  dict_ord[grupos_nomes[i]],
                                                                                  dict_ordem[grupos_nomes[i]],
                                                                                  k_max=k_maximo_por_grupo[grupos_nomes[i]]
                                                                                                            )
  features_novas_train = np.append(features_novas_train,features_novas_do_grupo,axis=1)
  modelos.append(Modelo)
  melhores_ks.append(k_melhor_grupo)

#deleção da coluna 'boba'
features_novas_train = np.delete(features_novas_train, 0, 1)

In [ ]:
features_novas_test = np.zeros((len(X_test),1))

for i in range(len(grupos)):
  features_novas_do_grupo = features_abstraidas_por_grupo_transform(
                                                                    X_test[grupos[i]],
                                                                    dict_num[grupos_nomes[i]],
                                                                    dict_cat[grupos_nomes[i]],
                                                                    dict_ord[grupos_nomes[i]],
                                                                    dict_ordem[grupos_nomes[i]],
                                                                    modelos[i][0],
                                                                    modelos[i][1],
                                                                    modelos[i][2],
                                                                    modelos[i][3]
                                                                    )
  features_novas_test = np.append(features_novas_test,features_novas_do_grupo,axis=1)
features_novas_test = np.delete(features_novas_test, 0, 1)

#Questão 2

##a) Aprimore o pipeline desenvolvido em aula para que seja possível comparar mais de uma técnica de modelagem (por exemplo, KNN e Árvore de decisão) usando um mesmo grid-search;

Importação de módulos que serão necessários

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

from sklearn.metrics import mean_squared_error

Uma maneira encontrada de utilizar vários estimadores seria através do mecanismo de herança dos estimadores do sklearn, junto da implementação dos métodos pedidos. Infelizmente desta forma o pipeline não é facilmente integrado com a noção de tunagem de hiperparâmetros dado um modelo em específico.

In [29]:
from sklearn.base import BaseEstimator
class RegSwitcher(BaseEstimator):

  def __init__(self, estimator=KNeighborsRegressor()):

      self.estimator = estimator


  def fit(self, X, y=None, **kwargs):
      self.estimator.fit(X, y)
      return self


  def predict(self, X, y=None):
      return self.estimator.predict(X)


  def predict_proba(self, X):
      return self.estimator.predict_proba(X)


  def score(self, X, y):
    return self.estimator.score(X, y)

Apenas para fins de exploração maior de possibilidades, adiciona-se uma redução de dimensionalidade antes da aplicação do estimador, para facilitar o treino. (escolhidos testes com 95%,90% e 85% da variância explicada pelos principais componentes).

Além disso vale a menção de testes com regressores de máquinas de vetor suporte, random forests e regressão linear simples

In [30]:
parameters_GSCV = {
    'PCA__n_components':[0.95,0.9,0.85],
    'model__estimator': [
                         SVR(),
                         KNeighborsRegressor(),
                         DecisionTreeRegressor(),
                         RandomForestRegressor(),
                         LinearRegression()
                         ]
}

pipe = Pipeline(steps = [
                 ('PCA',PCA(svd_solver = 'full')),
                 ('model',RegSwitcher()),
])

Gs = GridSearchCV(
    estimator = pipe,
    param_grid = parameters_GSCV,
    scoring = 'neg_root_mean_squared_error',
    cv = 3
)

Gs.fit(features_novas_train,y_train)
#Output deste fora mantido por conta de apresentar informações do modelo final

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel(

GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('PCA',
                                        PCA(copy=True, iterated_power='auto',
                                            n_components=None,
                                            random_state=None,
                                            svd_solver='full', tol=0.0,
                                            whiten=False)),
                                       ('model',
                                        RegSwitcher(estimator=KNeighborsRegressor(algorithm='auto',
                                                                                  leaf_size=30,
                                                                                  metric='minkowski',
                                                                                  metric_params=None,
                                                                                  n_job

In [31]:
Gs.best_estimator_
# 90% da variância explicada é preservada nestes componentes, melhor modelo é o de árvores

Pipeline(memory=None,
         steps=[('PCA',
                 PCA(copy=True, iterated_power='auto', n_components=0.9,
                     random_state=None, svd_solver='full', tol=0.0,
                     whiten=False)),
                ('model',
                 RegSwitcher(estimator=RandomForestRegressor(bootstrap=True,
                                                             ccp_alpha=0.0,
                                                             criterion='mse',
                                                             max_depth=None,
                                                             max_features='auto',
                                                             max_leaf_nodes=None,
                                                             max_samples=None,
                                                             min_impurity_decrease=0.0,
                                                             min_impurity_split=None,
                         

Como pode-se ver logo a seguir, a performance no conjunto de treino é medíocre, um pouco pior que o modelo muito mais simples apresentado em aula, mesmo com todas estas transformações e manipulações de valores. 

Hipóteses e soluções são propostas logo a seguir

In [32]:
-Gs.best_score_

39235.33611616312

##Questão 1

###b) Faça uma análise ilustrando a (possível) qualidade das variáveis propostas.

Para efetuar uma análise simples, vamos utilizar primeiramente o quesito feature importances do modelo de Random Forest, para isso mostra-se abaixo o acesso a essa característica.

In [33]:
Gs.best_estimator_[1].estimator.feature_importances_

array([0.61257803, 0.06542848, 0.06848894, 0.03714375, 0.01870512,
       0.01840162, 0.02537265, 0.01349098, 0.04218924, 0.01917945,
       0.01304475, 0.0145477 , 0.01520771, 0.01136016, 0.02486142])

Como pode-se ver uma grande parte da importância das features pode ser resumida na 1a componente da PCA ! 
Porém precisamos analisar realmente as variáveis novas criadas, portanto temos de treinar um novo modelo. Segue abaixo a criação, treinamento e estimativa da importância das features.

In [34]:
rf = RandomForestRegressor(random_state=42)
rf.fit(features_novas_train, y_train)

feat_df = pd.DataFrame(
    rf.feature_importances_, columns=['Importancias'])

feat_df=feat_df.iloc[:,0]
feat_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0     0.015515
1     0.006372
2     0.057792
3     0.006586
4     0.329490
5     0.025649
6     0.008108
7     0.005027
8     0.004095
9     0.005465
10    0.059239
11    0.003203
12    0.033313
13    0.028392
14    0.011621
15    0.006310
16    0.009424
17    0.012319
18    0.031438
19    0.009114
20    0.000788
21    0.008297
22    0.001337
23    0.009255
24    0.021820
25    0.007539
26    0.003635
27    0.010915
28    0.056624
29    0.069736
30    0.009348
31    0.001099
32    0.013482
33    0.005751
34    0.010842
35    0.005444
36    0.011853
37    0.015613
38    0.006613
39    0.002671
40    0.040218
41    0.001623
42    0.000834
43    0.001359
44    0.008213
45    0.006622
Name: Importancias, dtype: float64

Se formos olhar para o output do treinamento, notamos que as primeiras 13 features são do grupo 'garage', as 4 próximas são do grupo 'bsmt' e assim por diante, organizando esta informação toda num dicionário:

In [35]:
dict_ks_por_grupo = {
    'garage':melhores_ks[0],
    'bsmt': melhores_ks[1],
    'extras':melhores_ks[2],
    'local':melhores_ks[3],
    'infra':melhores_ks[4],
    'terreno_estilo':melhores_ks[5],
    'bathroom':melhores_ks[6],
    'bedroom':melhores_ks[7],
    'kitchen':melhores_ks[8],
    'roof':melhores_ks[9],
    'avgs':melhores_ks[10]
}

In [36]:
dict_ks_por_grupo

{'avgs': 2,
 'bathroom': 3,
 'bedroom': 6,
 'bsmt': 4,
 'extras': 2,
 'garage': 13,
 'infra': 5,
 'kitchen': 3,
 'local': 4,
 'roof': 2,
 'terreno_estilo': 2}

In [37]:
#aparentemente esta feature tem uma importância relativa de cerca de 0,8 %
feat_df.iloc[range(len(feat_df)-2,len(feat_df)-1)].sum()

0.00821336064181432

In [38]:
#o que torna pior que garage, que não se espera influência assim na prática
feat_df.iloc[range(0,melhores_ks[0])].sum()

0.5598529320994672

Conclusão: As features novas, sejam as explicativas ou os agrupamentos como um todo, não foram muito úteis ao modelo. 

A hipótese do porquê seria pelo fato que escolheram-se as features baseando na tentativa de manter a maior parte da entropia original possível, o que pode ser útil muitas vezes, porém deve-se lembrar que temos um objetivo de regressão de preço de casas, que deve-se dar mais importância para variáveis **correlacionadas com este target**, não necessariamente com o objetivo de manter toda a informação do dataset original.

Creio que a utilização de mais algumas análises nesta direção possam ser úteis, como a utilização de métricas como, da wikipédia ([https://pt.wikipedia.org/wiki/Entropia_cruzada](https://)): 

* *Na teoria da informação, a entropia cruzada se refere à diferença entre duas distribuições de probabilidade p (verdadeira) e q (estimada) sobre o mesmo conjunto de eventos. Na prática, a entropia cruzada mede o número médio de bits necessários para identificar um evento , se a codificação utilizada for otimizada para a distribuição de probabilidade estimada q , em vez de otimizada para a distribuição de probabilidade verdadeira p .*

Escolhendo os números de clusters de forma que aqueles que possum maior relação (nesta noção de entropia cruzada) com o target recebam valores mais altos seria um horizonte exploratório bom.

Outra alternativa (ainda tratando de teoria da informação) seria em uma análise com base na informação mútua entre essas duas variáveis, ou uma abordagem baseada especificamente para a inclusão de variáveis novas em problemas de regressão como AIC e BIC (respectivamente Akaike e Bayes Information Cryterion)

Outras formas de análise, mais algorítimicas, com algumas inclusive baseadas em teorias dos jogos, estão expostas logo a seguir:

* **LOFO**: 
###Explicação & github: https://github.com/aerdem4/lofo-importance
###Exemplo: https://www.kaggle.com/aerdem4/optiver-lofo-feature-importance

* **Null importances**:
###Artigo explicando: https://academic.oup.com/bioinformatics/article/26/10/1340/193348
###Notebook com exemplo: https://www.kaggle.com/ogrellier/feature-selection-with-null-importances
* **Importância por permutação**: 
###Explicação: https://eli5.readthedocs.io/en/latest/blackbox/permutation_importance.html
* **Outros**:
###BorutaPy: https://www.kaggle.com/residentmario/automated-feature-selection-with-boruta
###BorutaShap: https://github.com/Ekeany/Boruta-Shap
###ShapHypetune: https://github.com/cerlymarco/shap-hypetune

#Questão 3
##a) Comparar vários modelos candidatos e escolher o melhor;

Embora não tenham sido muito bons , o pipeline anterior (junto das features novas) nos deu um resultado mediano, que pode com certeza ser melhorado com alguma tunagem dos hiperparâmetros do melhor modelo. Para isso, vamos manter a Pca anterior e escolher a melhor combinação de valores para o modelo de florestas

In [39]:
Gs.best_estimator_[0]

PCA(copy=True, iterated_power='auto', n_components=0.9, random_state=None,
    svd_solver='full', tol=0.0, whiten=False)

In [40]:
parametros_floresta = {
    'RF__max_depth':[5,10,15],
    'RF__max_features':['auto','sqrt','log2'],
    'RF__bootstrap':[True,False],
    'RF__max_samples':[0.5,0.75,1.0]
}

pipe_forest = Pipeline(steps = [
                        ('PCA',Gs.best_estimator_[0]),
                        ('RF',RandomForestRegressor())
])

GsRF = GridSearchCV(
    estimator = pipe_forest,
    param_grid = parametros_floresta,
    scoring = 'neg_root_mean_squared_error',
    cv = 5
    ,n_jobs=-1
)

In [41]:
GsRF.fit(features_novas_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('PCA',
                                        PCA(copy=True, iterated_power='auto',
                                            n_components=0.9, random_state=None,
                                            svd_solver='full', tol=0.0,
                                            whiten=False)),
                                       ('RF',
                                        RandomForestRegressor(bootstrap=True,
                                                              ccp_alpha=0.0,
                                                              criterion='mse',
                                                              max_depth=None,
                                                              max_features='auto',
                                                              max_leaf_nodes=None,
                                                         

In [42]:
GsRF.best_estimator_

Pipeline(memory=None,
         steps=[('PCA',
                 PCA(copy=True, iterated_power='auto', n_components=0.9,
                     random_state=None, svd_solver='full', tol=0.0,
                     whiten=False)),
                ('RF',
                 RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                       criterion='mse', max_depth=15,
                                       max_features='auto', max_leaf_nodes=None,
                                       max_samples=0.75,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=1, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       n_estimators=100, n_jobs=None,
                                       oob_score=False, random_state=None,
                                       verbose=0, w

#Questão 3
###b) Gerar os resultados do melhor modelo na base de teste e enviar para o Kaggle.

In [43]:
y_pred_test = GsRF.predict(features_novas_test)

#expectativa de erro
print(mean_squared_error(y_test,y_pred_test,squared=False))

38344.32080447204


In [ ]:
id = test['Id'].copy()

test.drop(drop,axis=1,inplace=True)

total_area = test['LotArea'] + test['1stFlrSF'] + test['2ndFlrSF']
test['AvgAbvGrLivArea'] = test['GrLivArea']/total_area
test['AvgRmsAbvGrdSize'] = test['TotRmsAbvGrd']/total_area
test['AvgBedroomAbvGrSize'] = test['BedroomAbvGr']/total_area

features_novas_test = np.zeros((len(test),1))

for i in range(len(grupos)):
  features_novas_do_grupo_test = features_abstraidas_por_grupo_transform(
                                                                    test[grupos[i]],
                                                                    dict_num[grupos_nomes[i]],
                                                                    dict_cat[grupos_nomes[i]],
                                                                    dict_ord[grupos_nomes[i]],
                                                                    dict_ordem[grupos_nomes[i]],
                                                                    modelos[i][0],
                                                                    modelos[i][1],
                                                                    modelos[i][2],
                                                                    modelos[i][3]
                                                                    )
  features_novas_test = np.append(features_novas_test,features_novas_do_grupo_test,axis=1)
features_novas_test = np.delete(features_novas_test, 0, 1)

In [45]:
y_preds = GsRF.predict(features_novas_test)
submissao = pd.DataFrame({
    'Id': id,
    'SalePrice': y_preds
})

submissao.to_csv('sub.csv',columns=['Id','SalePrice'],index=False)
#score no Kaggle = 0.19740

#Questão 4

##a) Identificar um possível problema com a métrica de erro usada (RMSE). Justificar com explicações, exemplos e análises;

##b) Apontar uma nova métrica que não tenha esse problema. Justificar com explicações, exemplos e análises.

a) Um possível problema com a métrica de RMSE seria o fato de, ao analisar sua formulação matemática, percebe-se que existe (como desajado) a noção de distância euclidiana. 

Um modelo que minimiza a distância euclideana está, como pode ser mostrado formalmente, aproximando suas previsões de modo a coincidir com a **média** da previsões reais. 

Isso pode ser desejável em muitas situações, especialmente em casos que a média é muito provável, como em distribuições normais, a mais presente na descrição de dados contínuos.

Entretanto, do ponto de vista prático do problema isso pode ser problemático. Imagenemos por exemplo que desejam-se realizar previsões que rendam mais lucro à empresa. Se o modelo acertar mais em casos em que os preços são muito altos, e permitir-se errar mais em casos em que os preços são menores, é capaz de refletir, em um dado cenário de modelo de negócios, em um maior lucro, dado que provavelmente algumas propriedades muito caras podem ter o seu preço reduzido em muitas vezes, atraindo mais clientes para sua compra.

Fazendo uma contrapartida, muitas vezes possíveis proprietários que possuem possibilidade de comprar tais casas podem não "sentir" muito a diferença se o preço estiver um pouco acima, logo isso pode não ter um efeito tão simples de se prever, e variar de caso a caso.

Um outro ponto de crítica da métrica seria a observação de que o erro é não linear, sendo que valores de erros que estão entre 0 e 1 em módulo são largamente ignorados pela métrica em face de erros maiores, o que é bom na maioria dos casos, mas vale ressaltar que isso depende da definição da unidade nesta escala, o que pode ser em dólares, milhares ou milhões de dólares, sendo assim necessário um cuidado extra na definição de unidade.

b) Uma métrica que não possui estes mesmo erros seria o erro absoluto média (sigla em inglês MAE). Ao contrário da RMSE, esta forma de conceitualização de erro, quando minimizado, converge para a mediana da distribuição de dados, o que pode ser igual a média em muitos casos (sendo assim ao menos tão útil quanto) porém mais afastada em outros.

No exemplo apontado logo acima, quando existem casas mais caras, a mediana é menor que a média, o que implica em um menor foco nas casas mais caras, o que pode ser desejável em casos de clientela composta de ampla maioria de pessoas com menor condição monetária, em que um erro de 10% é qualitativamente diferente de um erro de 10% para alguém com melhor condição financeira.